# Sloppily Applying VGG16 To Identifying Types of Distracted Drivers

## Splitting Into Train/Validation Data

In [ ]:
import os,sys

VALIDATION_PERCENTAGE = 0.1 # Probably too low, took a wild guess, didn't want to google

commands = []

for img_class in os.listdir():
    files = os.listdir(img_class)
    validation_list = files[:int(len(files)*VALIDATION_PERCENTAGE)]
    commands.append('for f in %s/{%s}; do mv $f ../valid/$f; done' % (img_class, ','.join(validation_list)))


print(str(commands))
for command in commands:
    os.system(command) # Note, I actually ran this part from the shell. Just donig it in Python in the notebook for completeness.

## VGG Setup

In [1]:
%matplotlib inline

In [2]:
path = "data/distracted_driver/"

In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [4]:
import utils; reload(utils)
from utils import plots

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 750 Ti (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [8]:
vgg = Vgg16()

40960/35363 [==================================] - 0s 


In [32]:
batch_size=10

In [12]:
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)

Found 20187 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.


In [13]:
vgg.finetune(batches)

#### Time to Train

In [14]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
20187/20187 [==============================] - 738s - loss: 1.9401 - acc: 0.5191 - val_loss: 0.4409 - val_acc: 0.8561


## Sample of Results

In [ ]:
batches = vgg.get_batches(path+'valid', batch_size=4)
imgs,labels = next(batches)
plots(imgs, titles=labels)
vgg.predict(imgs, True)

_I don't want to retrain the model for this notebook, but in a 9-image trial it accurately classified 8/9 images (according to my human judgement), which is better than I expected based upon the stats printed by vgg.fit (luck of the sample?)._

Note that I used the validation data for sampling results. I wanted to use the `test1` data but could not figure out how :).